In [1]:
import itertools
import pathlib
import pyam

import numpy as np
import pandas as pd

<IPython.core.display.Javascript object>

In [2]:
data_path = pathlib.Path('../data')

In [3]:
LABEL = '2015_soft_grassi'
pddata = pd.read_csv(data_path / f'analysis_{LABEL}_data.csv')
df = pyam.IamDataFrame(pddata)
df.load_meta(data_path / f'analysis_{LABEL}_meta.csv')

pyam - INFO: Running in a notebook, setting up a basic logging at level INFO
pyam.core - INFO: Reading meta indicators


# Computer Carbon Budgets

In [18]:
def make_carbon_budget(df, base, add):
    meta = (
        df
        .meta
        .query("Category in ['C1', 'C3']")
        [f'Interpolated Net-Zero Year for {base}{add}']
        .dropna()
        .astype(int)
        .reset_index()
    ) 
    data = (
        df
        .filter(region='World', variable=f'Cumulative {base} from 2020')
        .convert_unit('Mt CO2/yr', to='Gt CO2', factor=1e-3)
    )
    budgets = [(
        row['model'], 
        row['scenario'], 
        data.filter(
            model=row['model'], scenario=row['scenario'], 
            year=row[f'Interpolated Net-Zero Year for {base}{add}']
        ).timeseries().values[0][0]
    )
        for i, row in meta.iterrows()
    ]
    return pd.DataFrame(budgets, columns=['model', 'scenario', f'Carbon Budget for {base}{add}']).set_index(['model', 'scenario'])
        

In [19]:
adds = ['', ' - Reanalyzed', ' - Reanalyzed and Adjusted']
new_meta = pd.concat([make_carbon_budget(df, 'Emissions|CO2', add) for add in adds], axis=1)
df.set_meta(new_meta)

# Emission Reductions

In [6]:
def compute_reduction(df, variable, fromyear=2020, toyear=2030):
    data = (
        df
        .filter(variable=variable, year=[fromyear, toyear], region='World')
        .timeseries()
        .droplevel(['region', 'unit', 'variable'])
    )
    s = (data[fromyear] - data[toyear]) / data[fromyear] * 100
    s.name = f'2030 Emission Reductions for {variable}'
    return s

In [9]:
adds = ['', ' - Reanalyzed', ' - Reanalyzed and Adjusted']
gases = ['Emissions|CO2', 'Emissions|Kyoto Gases']
new_meta = pd.concat([
    compute_reduction(df, f'{gas}{add}')
    for gas, add in itertools.product(gases, adds)
], axis=1)
df.set_meta(new_meta)

In [11]:
def summarize(data, base, name):
    adds = ['', ' - Reanalyzed', ' - Reanalyzed and Adjusted']
    cols = [f'{base}{add}' for add in adds]
    headers = ['Original Pathways', 'Pathways with OSCAR Direct Effects', 'Pathways with OSCAR Direct & Indirect Effects']
    return (
        data
        .query("Category in ['C1', 'C3']")
        .replace({'Category': {'C1': '1.5C', 'C3': '2C'}})
        .rename(columns={c: h for c, h in zip(cols, headers)})
        [headers + ['Category']]
        .groupby('Category')
        .describe(percentiles=(0.25, 0.5, 0.75))
        .stack(level=-2)
        [['50%', '25%', '75%']]
        .astype(int)
        .apply(lambda col: f"{col['50%']} ({col['25%']}-{col['75%']})", axis=1)
        .to_frame(name=name)
        .loc[['1.5C', '2C'], headers, :]
    )

In [12]:
to_summarize = {
    'Carbon Budget for Emissions|CO2': 'Carbon Budget from 2020',
    '2030 Emission Reductions for Emissions|CO2': 'CO2 Emissions Reductions (2020-2030)',
    'Interpolated Net-Zero Year for Emissions|CO2': 'Net-zero CO2 Year',
    'Interpolated Net-Zero Year for Emissions|Kyoto Gases': 'Net-zero GHG Year',
}

In [22]:
table = pd.concat([
    summarize(df.meta, base, name)
    for base, name in to_summarize.items()
], axis=1).T
table.to_excel('mitigation_outcomes.xlsx')
table

Category                                          1.5C  \
                                     Original Pathways   
Carbon Budget from 2020                  550 (470-572)   
CO2 Emissions Reductions (2020-2030)        47 (41-57)   
Net-zero CO2 Year                     2055 (2049-2060)   
Net-zero GHG Year                     2070 (2062-2076)   

Category                                                                 \
                                     Pathways with OSCAR Direct Effects   
Carbon Budget from 2020                                   538 (463-564)   
CO2 Emissions Reductions (2020-2030)                         48 (37-56)   
Net-zero CO2 Year                                      2051 (2047-2054)   
Net-zero GHG Year                                      2067 (2059-2077)   

Category                                                                            \
                                     Pathways with OSCAR Direct & Indirect Effects   
Carbon Budget from 2020                                              525 (461-555)   
CO2 Emissions Reductions (2020-2030)                                    57 (49-62)   
Net-zero CO2 Year                                                 2046 (2043-2049)   
Net-zero GHG Year                                                 2059 (2054-2073)   

Category                                            2C  \
                                     Original Pathways   
Carbon Budget from 2020                  933 (776-999)   
CO2 Emissions Reductions (2020-2030)         21 (6-30)   
Net-zero CO2 Year                     2071 (2067-2079)   
Net-zero GHG Year                     2082 (2078-2087)   

Category                                                                 \
                                     Pathways with OSCAR Direct Effects   
Carbon Budget from 2020                                   901 (754-989)   
CO2 Emissions Reductions (2020-2030)                          18 (7-29)   
Net-zero CO2 Year                                      2068 (2063-2075)   
Net-zero GHG Year                                      2080 (2074-2085)   

Category                                                                            
                                     Pathways with OSCAR Direct & Indirect Effects  
Carbon Budget from 2020                                              898 (745-979)  
CO2 Emissions Reductions (2020-2030)                                    26 (12-36)  
Net-zero CO2 Year                                                 2062 (2056-2069)  
Net-zero GHG Year                                                 2080 (2071-2085)